In [149]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split, cross_val_score, learning_curve, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier

In [150]:
data = pd.read_csv('/kaggle/input/my-churn-prediction-dataset/bank-customer-churn-prediction-spark4ai/train.csv')
print(data.shape)
data.head()

(15000, 14)


,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15635502.0,Pirozzi,632.0,France,Female,63.0,5.0,0.00,2.0,0.0,1.0,140780.80,0.0
1,1,15595440.0,Horsley,730.0,Germany,Female,35.0,5.0,182532.23,2.0,1.0,1.0,136491.32,0.0
2,2,15803566.0,Zito,598.0,Spain,Male,45.0,2.0,94049.92,1.0,1.0,1.0,131718.12,0.0
3,3,15693906.0,Iloerika,678.0,France,Female,38.0,9.0,0.00,2.0,1.0,1.0,164768.82,0.0
4,4,15679531.0,K'ung,710.0,France,Female,32.0,4.0,0.00,2.0,0.0,0.0,100187.43,0.0


In [151]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               15000 non-null  int64  
 1   CustomerId       15000 non-null  float64
 2   Surname          15000 non-null  object 
 3   CreditScore      15000 non-null  float64
 4   Geography        15000 non-null  object 
 5   Gender           15000 non-null  object 
 6   Age              15000 non-null  float64
 7   Tenure           15000 non-null  float64
 8   Balance          15000 non-null  float64
 9   NumOfProducts    15000 non-null  float64
 10  HasCrCard        15000 non-null  float64
 11  IsActiveMember   15000 non-null  float64
 12  EstimatedSalary  15000 non-null  float64
 13  Exited           15000 non-null  float64
dtypes: float64(10), int64(1), object(3)
memory usage: 1.6+ MB


In [152]:
data['Exited'].value_counts()

Exited
0.0    11937
1.0     3063
Name: count, dtype: int64

In [153]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               15000 non-null  int64  
 1   CustomerId       15000 non-null  float64
 2   Surname          15000 non-null  object 
 3   CreditScore      15000 non-null  float64
 4   Geography        15000 non-null  object 
 5   Gender           15000 non-null  object 
 6   Age              15000 non-null  float64
 7   Tenure           15000 non-null  float64
 8   Balance          15000 non-null  float64
 9   NumOfProducts    15000 non-null  float64
 10  HasCrCard        15000 non-null  float64
 11  IsActiveMember   15000 non-null  float64
 12  EstimatedSalary  15000 non-null  float64
 13  Exited           15000 non-null  float64
dtypes: float64(10), int64(1), object(3)
memory usage: 1.6+ MB


In [154]:
cat_features = data[['Gender', 'Geography']]

cat_features.head()

,Gender,Geography
0,Female,France
1,Female,Germany
2,Male,Spain
3,Female,France
4,Female,France


In [155]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df_cat = cat_features.apply(le.fit_transform)
df_cat.head()



,Gender,Geography
0,0,0
1,0,1
2,1,2
3,0,0
4,0,0


In [156]:
num_features = data[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']]
finaldf = pd.merge(num_features, df_cat, left_index=True, right_index=True)
finaldf.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender,Geography
0,632.0,63.0,5.0,0.00,2.0,0.0,1.0,140780.80,0.0,0,0
1,730.0,35.0,5.0,182532.23,2.0,1.0,1.0,136491.32,0.0,0,1
2,598.0,45.0,2.0,94049.92,1.0,1.0,1.0,131718.12,0.0,1,2
3,678.0,38.0,9.0,0.00,2.0,1.0,1.0,164768.82,0.0,0,0
4,710.0,32.0,4.0,0.00,2.0,0.0,0.0,100187.43,0.0,0,0


In [157]:
aggs = {
    'Age': ['min', 'max', 'mean'],
    'Balance': ['min', 'max', 'mean', 'sum'],
    'NumOfProducts': ['mean', 'sum'],
    'IsActiveMember': ['min', 'max', 'mean', 'sum'],
    'CreditScore': ['min', 'max', 'mean'],
    'EstimatedSalary': ['min', 'max', 'mean', 'sum'],
    'id': 'count'
}

data['Gender'] = df_cat['Gender']
data['Geography'] = df_cat['Geography']

df_grps = data.groupby(['CustomerId', 'Surname', 'Geography', 'Gender']).agg(aggs).reset_index()
df_grps.columns = [''.join(col) for col in df_grps.columns.values]
print("Shape of df_grps:", df_grps.shape)
print(df_grps.head())

# Second aggregation
aggs2 = {
    'Balance': ['min', 'max', 'mean', 'sum'],
    'NumOfProducts': ['mean', 'sum'],
    'IsActiveMember': ['min', 'max', 'mean', 'sum'],
    'CreditScore': ['min', 'max', 'mean'],
    'EstimatedSalary': ['min', 'max', 'mean', 'sum'],
    'id': 'count'
}

df_grps1 = data.groupby(['CustomerId', 'Surname', 'Age', 'Gender']).agg(aggs2).reset_index()
df_grps1.columns = ['grps1'.join(col) for col in df_grps1.columns.values]
df_grps1 = df_grps1.rename(columns={
    'CustomerIdgrps1': 'CustomerId',
    'Surnamegrps1': 'Surname',
    'Agegrps1': 'Age',
    'Gendergrps1': 'Gender'
})
print("\nShape of df_grps1:", df_grps1.shape)
print(df_grps1.head())

# Merge the new features with the original dataset
data_enriched = data.merge(df_grps, on=['CustomerId', 'Surname', 'Geography', 'Gender'], how='left')
data_enriched = data_enriched.merge(df_grps1, on=['CustomerId', 'Surname', 'Age', 'Gender'], how='left')


print("\nShape of data_enriched:", data_enriched.shape)
data_enriched.head()

Shape of df_grps: (14965, 25)
   CustomerId      Surname  Geography  Gender  Agemin  Agemax  Agemean  \
0  15565796.0         Teng          1       0    26.0    26.0     26.0   
1  15565879.0  Chukwujekwu          1       1    35.0    35.0     35.0   
2  15565996.0        T'ien          1       1    45.0    45.0     45.0   
3  15566156.0      Okwuoma          1       1    40.0    40.0     40.0   
4  15566211.0     Chidimma          0       1    36.0    36.0     36.0   

   Balancemin  Balancemax  Balancemean  ...  IsActiveMembermean  \
0   111097.85   111097.85    111097.85  ...                 1.0   
1   142645.64   142645.64    142645.64  ...                 0.0   
2   115084.53   115084.53    115084.53  ...                 1.0   
3   121345.88   121345.88    121345.88  ...                 1.0   
4        0.00        0.00         0.00  ...                 1.0   

   IsActiveMembersum  CreditScoremin  CreditScoremax  CreditScoremean  \
0                1.0           753.0           75

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,...,IsActiveMembergrps1mean,IsActiveMembergrps1sum,CreditScoregrps1min,CreditScoregrps1max,CreditScoregrps1mean,EstimatedSalarygrps1min,EstimatedSalarygrps1max,EstimatedSalarygrps1mean,EstimatedSalarygrps1sum,idgrps1count
0,0,15635502.0,Pirozzi,632.0,0,0,63.0,5.0,0.00,2.0,...,1.0,1.0,632.0,632.0,632.0,140780.80,140780.80,140780.80,140780.80,1
1,1,15595440.0,Horsley,730.0,1,0,35.0,5.0,182532.23,2.0,...,1.0,1.0,730.0,730.0,730.0,136491.32,136491.32,136491.32,136491.32,1
2,2,15803566.0,Zito,598.0,2,1,45.0,2.0,94049.92,1.0,...,1.0,1.0,598.0,598.0,598.0,131718.12,131718.12,131718.12,131718.12,1
3,3,15693906.0,Iloerika,678.0,0,0,38.0,9.0,0.00,2.0,...,1.0,1.0,678.0,678.0,678.0,164768.82,164768.82,164768.82,164768.82,1
4,4,15679531.0,K'ung,710.0,0,0,32.0,4.0,0.00,2.0,...,0.0,0.0,710.0,710.0,710.0,100187.43,100187.43,100187.43,100187.43,1


In [158]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

columns_to_scale = ['CreditScore', 'Age', 'EstimatedSalary', 'Balance']


X = data_enriched.drop(['Exited', 'Surname', 'CustomerId'], axis=1)
y = data_enriched['Exited']

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [159]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE(k_neighbors=5)
X_smote, y_smote = oversample.fit_resample(X_train, y_train)
X_train, y_train = X_smote, y_smote

In [160]:
y_train.value_counts()

Exited
0.0    9551
1.0    9551
Name: count, dtype: int64

In [161]:
# rf = RandomForestClassifier(random_state=34)
# rf.fit(X_train,y_train)

In [162]:
# from sklearn.metrics import accuracy_score

# preds = rf.predict(X_test)
# print(accuracy_score(preds,y_test))

In [163]:
model = RandomForestClassifier()

In [164]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
print(f'Cross-validation scores: {scores}')
print(f'Mean cross-validation score: {scores.mean()}')

Cross-validation scores: [0.89166667 0.893      0.89433333 0.88766667 0.88733333]
Mean cross-validation score: 0.8908000000000001
